In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
!pip install pyaml

In [ ]:
!pip install pillow

In [ ]:
import os
import random
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
import yaml
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image


from ultralytics import YOLO
import torch
from PIL import Image
from torchvision import transforms



Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Dataset paths
data_paths = {
    'background': "/content/drive/MyDrive/garlic_yolo_sort/back_ground",
    'damaged': "/content/drive/MyDrive/garlic_yolo_sort/damaged_garlic",
    'healthy': "/content/drive/MyDrive/garlic_yolo_sort/healthy_garlic"
}

# Collect all images with their labels
def load_dataset():
    data = []
    class_ids = {'background': 0, 'damaged': 1, 'healthy': 2}

    for class_name, path in data_paths.items():
        class_id = class_ids[class_name]
        for img_file in os.listdir(path):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                data.append((os.path.join(path, img_file), class_id))

    random.shuffle(data)
    return data

# Load and split dataset
full_dataset = load_dataset()
train_data, test_data = train_test_split(full_dataset, test_size=0.3, random_state=42)

print(f"Total samples: {len(full_dataset)}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# Custom Dataset Class
class GarlicDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform or transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        return img, label

# Create datasets and dataloaders
train_dataset = GarlicDataset(train_data)
test_dataset = GarlicDataset(test_data)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=2)

Total samples: 2364
Training samples: 1654
Test samples: 710


In [ ]:
import os
import random
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torchvision import transforms, models
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
tf.random.set_seed(42)

# Dataset paths
data_paths = {
    'background': "/content/drive/MyDrive/garlic_yolo_sort/back_ground",
    'damaged': "/content/drive/MyDrive/garlic_yolo_sort/damaged_garlic",
    'healthy': "/content/drive/MyDrive/garlic_yolo_sort/healthy_garlic"
}

# Collect all images with their labels
def load_dataset():
    data = []
    class_ids = {'background': 0, 'damaged': 1, 'healthy': 2}

    for class_name, path in data_paths.items():
        class_id = class_ids[class_name]
        for img_file in os.listdir(path):
            if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                data.append((os.path.join(path, img_file), class_id))

    random.shuffle(data)
    return data

# Load and split dataset
full_dataset = load_dataset()
train_data, test_data = train_test_split(full_dataset, test_size=0.3, random_state=42)

print(f"Total samples: {len(full_dataset)}")
print(f"Training samples: {len(train_data)}")
print(f"Test samples: {len(test_data)}")

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Custom Dataset Class
class GarlicDataset(torch.utils.data.Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

# Create datasets and dataloaders
train_dataset = GarlicDataset(train_data, transform=transform)
test_dataset = GarlicDataset(test_data, transform=transform)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=2)

Total samples: 2364
Training samples: 1654
Test samples: 710


In [ ]:

# Create a Keras-compatible model (MobileNetV2 for RPi)
def create_keras_model(num_classes=3):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = True

    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=True)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Convert PyTorch DataLoader to Keras-compatible format
def dataloader_to_keras(dataloader):
    images = []
    labels = []
    for batch in dataloader:
        batch_images, batch_labels = batch
        batch_images = batch_images.numpy().transpose(0, 2, 3, 1)
        batch_labels = batch_labels.numpy()

        # Denormalize images
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        batch_images = std * batch_images + mean
        batch_images = np.clip(batch_images, 0, 1)

        images.append(batch_images)
        labels.append(batch_labels)

    return np.concatenate(images), np.concatenate(labels)

# Prepare data for Keras
x_train, y_train = dataloader_to_keras(train_loader)
x_test, y_test = dataloader_to_keras(test_loader)

# Create and train Keras model with callbacks
keras_model = create_keras_model()

# Callbacks to save best model and early stopping
checkpoint = ModelCheckpoint(
    'best_garlic_classifier.h5',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Train the model
history = keras_model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint, early_stopping]
)

# Load the best saved model
best_model = load_model('best_garlic_classifier.h5')

# Evaluate the best model
test_loss, test_acc = best_model.evaluate(x_test, y_test)
print(f"\nBest Model Test Accuracy: {test_acc*100:.2f}%")

# Save the final model (optional)
keras_model.save('final_garlic_classifier.h5')

# Convert best model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
tflite_model = converter.convert()

# Save the TFLite model
with open('best_garlic_classifier.tflite', 'wb') as f:
    f.write(tflite_model)
print("Best TFLite model saved as 'best_garlic_classifier.tflite'")

# Prediction function
def predict_image(model, image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((224, 224))

    # Preprocess image
    img_array = np.array(img) / 255.0
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img_array = (img_array - mean) / std
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    predictions = model.predict(img_array)
    class_id = np.argmax(predictions[0])
    confidence = np.max(predictions[0])

    class_names = ['background', 'damaged', 'healthy']
    return class_names[class_id], confidence

# Test prediction
sample_image = test_data[0][0]
pred_class, pred_conf = predict_image(best_model, sample_image)
print(f"\nSample test image: {sample_image}")
print(f"Predicted: {pred_class} with confidence {pred_conf:.2f}")

Epoch 1/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.8977 - loss: 0.3267
Epoch 1: val_accuracy improved from -inf to 0.30704, saving model to best_garlic_classifier.h5


52/52 ━━━━━━━━━━━━━━━━━━━━ 547s 10s/step - accuracy: 0.8986 - loss: 0.3250 - val_accuracy: 0.3070 - val_loss: 19.5507
Epoch 2/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9650 - loss: 0.0940
Epoch 2: val_accuracy did not improve from 0.30704
52/52 ━━━━━━━━━━━━━━━━━━━━ 507s 10s/step - accuracy: 0.9653 - loss: 0.0933 - val_accuracy: 0.2901 - val_loss: 22.1074
Epoch 3/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9948 - loss: 0.0227
Epoch 3: val_accuracy did not improve from 0.30704
52/52 ━━━━━━━━━━━━━━━━━━━━ 480s 9s/step - accuracy: 0.9948 - loss: 0.0227 - val_accuracy: 0.2901 - val_loss: 22.3129
Epoch 4/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9869 - loss: 0.0399
Epoch 4: val_accuracy improved from 0.30704 to 0.33662, saving model to best_garlic_classifier.h5


52/52 ━━━━━━━━━━━━━━━━━━━━ 507s 9s/step - accuracy: 0.9870 - loss: 0.0398 - val_accuracy: 0.3366 - val_loss: 21.8018
Epoch 5/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9975 - loss: 0.0185
Epoch 5: val_accuracy improved from 0.33662 to 0.39577, saving model to best_garlic_classifier.h5


52/52 ━━━━━━━━━━━━━━━━━━━━ 482s 9s/step - accuracy: 0.9973 - loss: 0.0191 - val_accuracy: 0.3958 - val_loss: 21.7391
Epoch 6/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9578 - loss: 0.1481
Epoch 6: val_accuracy did not improve from 0.39577
52/52 ━━━━━━━━━━━━━━━━━━━━ 526s 9s/step - accuracy: 0.9582 - loss: 0.1469 - val_accuracy: 0.2901 - val_loss: 19.1269
Epoch 7/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9827 - loss: 0.0622
Epoch 7: val_accuracy did not improve from 0.39577
52/52 ━━━━━━━━━━━━━━━━━━━━ 483s 9s/step - accuracy: 0.9828 - loss: 0.0619 - val_accuracy: 0.3127 - val_loss: 11.6426
Epoch 8/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9966 - loss: 0.0100
Epoch 8: val_accuracy improved from 0.39577 to 0.48873, saving model to best_garlic_classifier.h5


52/52 ━━━━━━━━━━━━━━━━━━━━ 492s 9s/step - accuracy: 0.9966 - loss: 0.0101 - val_accuracy: 0.4887 - val_loss: 18.7775
Epoch 9/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 0.9970 - loss: 0.0105
Epoch 9: val_accuracy improved from 0.48873 to 0.58169, saving model to best_garlic_classifier.h5


...

52/52 ━━━━━━━━━━━━━━━━━━━━ 505s 9s/step - accuracy: 1.0000 - loss: 3.4774e-05 - val_accuracy: 0.6972 - val_loss: 8.8536
Epoch 19/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 1.0000 - loss: 3.0313e-05
Epoch 19: val_accuracy improved from 0.69718 to 0.71690, saving model to best_garlic_classifier.h5


52/52 ━━━━━━━━━━━━━━━━━━━━ 494s 10s/step - accuracy: 1.0000 - loss: 3.0278e-05 - val_accuracy: 0.7169 - val_loss: 7.5414
Epoch 20/20
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8s/step - accuracy: 1.0000 - loss: 2.6785e-05
Epoch 20: val_accuracy improved from 0.71690 to 0.73239, saving model to best_garlic_classifier.h5


52/52 ━━━━━━━━━━━━━━━━━━━━ 486s 9s/step - accuracy: 1.0000 - loss: 2.6757e-05 - val_accuracy: 0.7324 - val_loss: 6.3145
Restoring model weights from the end of the best epoch: 20.


23/23 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.7583 - loss: 5.6564



Best Model Test Accuracy: 73.24%
Saved artifact at '/tmp/tmpa70_hynk'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  136548756602512: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755458192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755459152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755458000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755458960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755459920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755457808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548622874960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755457040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136548755458384: TensorSpec(shape=(), dtype=tf.re